In [26]:
from bigfeat_base import BigFeat
from local_utils import *
import pandas as pd
from sklearn.preprocessing import LabelEncoder

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, make_scorer
import os
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import time
from sklearn.metrics import f1_score
from autofeat import AutoFeatClassifier
import matplotlib.pyplot as plt
import os


In [3]:
baseDir=r'/home/ubuntu/BigFeat-master/BigFeat-master/bigfeat/AutoMl_datasets'
#baseDir=r'C:\Users\kayak\Downloads\BigFeat-master\BigFeat-master\bigfeat\AutoMl_datasets\New folder'

datasets=os.listdir(baseDir)
ml_models={"Logistic_Regression":LogisticRegression(),
           "AdaBoostClassifier":AdaBoostClassifier(n_estimators=100, random_state=0),
           "DecisionTreeClassifier":DecisionTreeClassifier(random_state=0, max_depth=5),
           "ExtraTreesClassifier":ExtraTreesClassifier(n_estimators=100, random_state=0),
           "KNNClassifier":KNeighborsClassifier(n_neighbors=3),
           "MLP":MLPClassifier(random_state=1, max_iter=300),
           "RandomForestClassifier":RandomForestClassifier(max_depth=2, random_state=0),
           "SVM":svm.SVC(kernel='linear',probability=True),
           "GradientBoostingClassifier":GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0)
          }
    


In [4]:
data_indices={}

In [5]:

iterables = [[i.split('.')[0] for i in datasets], ["BigFeat","AutoFeat"],["Original", "Generated"],['F1_score','AUC']]

index = pd.MultiIndex.from_product(iterables, names=["DataSet","Framework","Type","Evaluation"])

dataframe= pd.DataFrame(columns=[i for i in ml_models], index=index)
    
dataframe


Logistic_Regression AdaBoostClassifier  \
DataSet Framework Type      Evaluation                                          
phoneme BigFeat   Original  F1_score                   NaN                NaN   
                            AUC                        NaN                NaN   
                  Generated F1_score                   NaN                NaN   
                            AUC                        NaN                NaN   
        AutoFeat  Original  F1_score                   NaN                NaN   
                            AUC                        NaN                NaN   
                  Generated F1_score                   NaN                NaN   
                            AUC                        NaN                NaN   

                                       DecisionTreeClassifier  \
DataSet Framework Type      Evaluation                          
phoneme BigFeat   Original  F1_score                      NaN   
                            AUC                           NaN   
                  Generated F1_score                      NaN   
                            AUC                           NaN   
        AutoFeat  Original  F1_score                      NaN   
                            AUC                           NaN   
                  Generated F1_score                      NaN   
                            AUC                           NaN   

                                       ExtraTreesClassifier KNNClassifier  \
DataSet Framework Type      Evaluation                                      
phoneme BigFeat   Original  F1_score                    NaN           NaN   
                            AUC                         NaN           NaN   
                  Generated F1_score                    NaN           NaN   
                            AUC                         NaN           NaN   
        AutoFeat  Original  F1_score                    NaN           NaN   
                            AUC                         NaN           NaN   
                  Generated F1_score                    NaN           NaN   
                            AUC                         NaN           NaN   

                                        MLP RandomForestClassifier  SVM  \
DataSet Framework Type      Evaluation                                    
phoneme BigFeat   Original  F1_score    NaN                    NaN  NaN   
                            AUC         NaN                    NaN  NaN   
                  Generated F1_score    NaN                    NaN  NaN   
                            AUC         NaN                    NaN  NaN   
        AutoFeat  Original  F1_score    NaN                    NaN  NaN   
                            AUC         NaN                    NaN  NaN   
                  Generated F1_score    NaN                    NaN  NaN   
                            AUC         NaN                    NaN  NaN   

                                       GradientBoostingClassifier  
DataSet Framework Type      Evaluation                             
phoneme BigFeat   Original  F1_score                          NaN  
                            AUC                               NaN  
                  Generated F1_score                          NaN  
                            AUC                               NaN  
        AutoFeat  Original  F1_score                          NaN  
                            AUC                               NaN  
                  Generated F1_score                          NaN  
                            AUC                               NaN

In [7]:
#dataframe.loc[('phoneme','BigFeat','Original')]

In [8]:
def getData(directory):
    data=pd.read_csv(directory)
    data=data.dropna() #  For the simplicity, drop the null values from dataset
    y=data.Label
    X=data.drop('Label',axis=1)
    return X,y

In [9]:
def getBarPlot(dataframe,evaluation_metric):

    N=len(dataframe.columns)

    x1=dataframe.loc[('phoneme','BigFeat','Original',evaluation_metric)].tolist()
    x2=dataframe.loc[('phoneme','BigFeat','Generated',evaluation_metric)].tolist()
    x3=dataframe.loc[('phoneme','AutoFeat','Generated',evaluation_metric)].tolist()

    for j in range(N):
    
        fig = plt.figure()
        ax = fig.add_axes([0,0,1,1])
        features=['Original','BigFeat','AutoFeat']
        values=[x1[j],x2[j],x3[j]]
    
        plt.ylabel(evaluation_metric + ' score')
        plt.xlabel('Features')
        plt.title(dataframe.columns[j])
    
        bars=plt.bar(features,values,color = 'b', width = 0.3)
        
        
        for bar in bars:
            yval = bar.get_height()
            plt.text(bar.get_x(), round(yval,5)+.010, round(yval,5))
            
        plt.show()
    

In [19]:
def dataforSAFE(data_indices):
    parent_dir=r'/home/ubuntu/BigFeat-master/BigFeat-master/bigfeat/AutoMl_datasets'
    data_folder=os.listdir(r'/home/ubuntu/BigFeat-master/BigFeat-master/bigfeat/AutoMl_datasets')
    if not os.path.exists(r'/home/ubuntu/BigFeat-master/BigFeat-master/bigfeat/AutoMl_datasets/SAFE_dataset'):
        os.mkdir(r'/home/ubuntu/BigFeat-master/BigFeat-master/bigfeat/AutoMl_datasets/SAFE_dataset')
    directory_SAFE=r'/home/ubuntu/BigFeat-master/BigFeat-master/bigfeat/AutoMl_datasets/SAFE_dataset'
    for data in data_folder:
        directory=f"{directory_SAFE}/{data.split('.')[0]}"
        data_name=data.split('.')[0]
        data_frame=pd.read_csv(f'{parent_dir}/{data}')
        slice_1=data_indices[data_name][0]
        slice_2=data_indices[data_name][1]
                            
        if len(slice_1)>len(slice_2):
            data_train=data_frame.iloc[slice_1]
            data_test=data_frame.iloc[slice_2]
        else:
            data_train=data_frame.iloc[slice_2]
            data_test=data_frame.iloc[slice_1]
            
        data_train.to_csv(f'{directory}_train.csv')
        data_test.to_csv(f'{directory}_test.csv')
                

In [11]:
def mlPipeline (X,y,ml_models,data_name,Framework):
    
    # This data's for significance analysis
    data_Container={}
    
    
    if Framework=='BigFeat':
        data_BigFeat={}
        data_Original={}
        
        #Feature generation
        X_train, X_test, y_train, y_test,indices_train,indices_test = train_test_split(X, y,np.arange(len(X)), test_size=0.33, random_state=42)
        if data_indices.get(data_name)==None:
            data_indices[data_name]=[]     
            data_indices[data_name].append(indices_train)
            data_indices[data_name].append(indices_test)
        feature_generation=BigFeat()
        X_train_tr=feature_generation.fit(X_train,y_train)
    
    
    
        for i in ml_models:
            
            #Generated features
            
            model=ml_models[i]
            model.fit(X_train_tr,y_train)
          
            
            if len(y.unique())>2:   # For multiclass classification
                y_pred=model.predict_proba(feature_generation.transform(X_test))
                data_BigFeat[i]=y_pred
                try: 
                    score_auc=roc_auc_score(y_test,y_pred,multi_class="ovr")
                    score_f1=f1_score(y_test, y_pred, average='weighted')
                except ValueError:
                    pass
               
                dataframe.loc[(data_name,Framework,"Generated","AUC")][i]=score_auc
                dataframe.loc[(data_name,Framework,"Generated","F1_score")][i]=score_f1
                #dataframe.loc[(data_name,"Generated")]['Framework']=Framework
            else :
                y_pred=model.predict(feature_generation.transform(X_test))
                data_BigFeat[i]=y_pred
                try:
                    score_auc=roc_auc_score(y_test,y_pred)
                    score_f1=f1_score(y_test, y_pred)
                except ValueError:
                    pass
                dataframe.loc[(data_name,Framework,"Generated",'AUC')][i]=score_auc
                dataframe.loc[(data_name,Framework,"Generated",'F1_score')][i]=score_f1
                #dataframe.loc[(data_name,"Generated")]['Framework']=Framework
                data_Container[Framework]=data_BigFeat
        
            
    
            #originial Feature
        
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
            model=ml_models[i]
            model.fit(X_train,y_train)
            
            
            
            if len(y.unique())>2:   # For multiclass classification
                
                y_pred=model.predict_proba(X_test)
                data_Original[i]=y_test
                try:        # if tartget variable extremly unbias 
                    score_auc=roc_auc_score(y_test,y_pred,multi_class="ovr")
                    score_f1=f1_score(y_test, y_pred, average='weighted')
                except ValueError:
                    pass
                
                dataframe.loc[(data_name,Framework,"Original","AUC")][i]=score_auc
                dataframe.loc[(data_name,Framework,"Original","F1_score")][i]=score_f1
                #dataframe.loc[(data_name,Framework,"Original")]['Framework']=Framework
            else :
                y_pred=model.predict(X_test)
                data_Original[i]=y_test
                try:
                    score_auc=roc_auc_score(y_test,y_pred)
                    score_f1=f1_score(y_test, y_pred)
                except ValueError:
                    pass
                dataframe.loc[(data_name,Framework,"Original","AUC")][i]=score_auc
                dataframe.loc[(data_name,Framework,"Original","F1_score")][i]=score_f1
               # dataframe.loc[(data_name,"Original")]['Framework']=Framework
            
            
            data_Container['Original']=data_Original
    
    if Framework=='AutoFeat':
        data_AutoFeat={}
        
        #Feature generation
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
            
        feature_generation=AutoFeatClassifier(verbose=0, feateng_steps=2,featsel_runs=5)
        X_train_tr=feature_generation.fit_transform(X_train,y_train)
    
        
        
        for i in ml_models:
        
            #Generated features
            
            model=ml_models[i]
            model.fit(X_train_tr,y_train)
          
            
            if len(y.unique())>2:   # For multiclass classification
                y_pred=model.predict_proba(feature_generation.transform(X_test))
                data_AutoFeat[i]=y_pred
                try: 
                    score_auc=roc_auc_score(y_test,y_pred,multi_class="ovr")
                    score_f1=f1_score(y_test, y_pred, average='weighted')
                except ValueError:
                    pass
                
                dataframe.loc[(data_name,Framework,"Generated","AUC")][i]=score_auc
                dataframe.loc[(data_name,Framework,"Generated","F1_score")][i]=score_f1
                #dataframe.loc[(data_name,"Generated")]['Framework']=Framework
            else :
                y_pred=model.predict(feature_generation.transform(X_test))
                data_AutoFeat[i]=y_pred
                try:
                    score_auc=roc_auc_score(y_test,y_pred)
                    score_f1=f1_score(y_test, y_pred)
                except ValueError:
                    pass
                dataframe.loc[(data_name,Framework,"Generated","AUC")][i]=score_auc
                dataframe.loc[(data_name,Framework,"Generated","F1_score")][i]=score_f1
                #dataframe.loc[(data_name,"Generated")]['Framework']=Framework
                data_Container[Framework]=data_AutoFeat
        
            #originial Feature
        
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
            model=ml_models[i]
            model.fit(X_train,y_train)
            
            
            if len(y.unique())>2:   # For multiclass classification
                
                y_pred=model.predict_proba(X_test)
                try:        # if tartget variable extremly unbias 
                    score_auc=roc_auc_score(y_test,y_pred,multi_class="ovr")
                    score_f1=f1_score(y_test, y_pred, average='weighted')
                    
                except ValueError:
                    pass
                dataframe.loc[(data_name,Framework,"Original","AUC")][i]=score_auc
                dataframe.loc[(data_name,Framework,"Original","F1_score")][i]=score_f1
                #dataframe.loc[(data_name,"Original")]['Framework']=Framework
            else :
                y_pred=model.predict(X_test)
                try:
                    score_auc=roc_auc_score(y_test,y_pred)
                    score_f1=f1_score(y_test, y_pred)
                except ValueError:
                    pass
                dataframe.loc[(data_name,Framework,"Original","AUC")][i]=score_auc
                dataframe.loc[(data_name,Framework,"Original","F1_score")][i]=score_f1
                #dataframe.loc[(data_name,"Original")]['Framework']=Framework
                
    #createExcel(data_Container,data_name,ml_models)           

In [12]:
def startPipeline(directory,datasets,ml_models,framework):
    
    for dataset in datasets:
        start=time.time()
        X,y=getData(f'{directory}/{dataset}')
        
        mlPipeline(X,y,ml_models,dataset.split('.')[0],framework)
        stop=time.time()
        print("{} seconds taken for the dataset {}".format(stop-start,dataset))
    return dataframe

In [13]:
def createExcel(data,data_name,ml_models):
    
    parent_dir=r'/home/ubuntu/BigFeat-master/bigfeat/Excels'
    path = os.path.join(parent_dir, data_name)
    
    if not os.path.exists(path):
        os.makedirs(path)

    BigFeat=data.get('BigFeat')
    Original=data.get('Original')
    AutoFeat=data.get('AutoFeat')
    
    for model in ml_models:
        dataFrame=pd.DataFrame(columns=['Original','BigFeat','AutoFeat'])
        if BigFeat !=None:
            if not os.path.exists(f'{path}/{model}.xlsx'):
                dataFrame['Original']=Original[model]
                dataFrame['BigFeat']=BigFeat[model]
                dataFrame.to_excel(f'{path}/{model}.xlsx')
            else:
                dataFrame=pd.read_excel(f'{path}/{model}.xlsx')
                dataFrame['Original']=Original[model]
                dataFrame['BigFeat']=BigFeat[model]
                dataFrame.to_excel(f'{path}/{model}.xlsx')
        if AutoFeat != None:
            
            if not os.path.exists(f'{path}/{model}.xlsx'):
                dataFrame['AutoFeat']=AutoFeat[model]
                dataFrame.to_excel(f'{path}/{model}.xlsx')
            else:
                dataFrame=pd.read_excel(f'{path}/{model}.xlsx')
                dataFrame['AutoFeat']=AutoFeat[model]
                dataFrame.to_excel(f'{path}/{model}.xlsx')


In [15]:
startPipeline(baseDir,datasets,ml_models,'BigFeat')

C:\Users\kayak\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\kayak\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


19.002092123031616 seconds taken for the dataset phoneme.csv


Logistic_Regression AdaBoostClassifier  \
DataSet Framework Type      Evaluation                                          
phoneme BigFeat   Original  F1_score              0.837716               0.88   
                            AUC                    0.67018            0.78058   
                  Generated F1_score              0.835698           0.888627   
                            AUC                   0.659936           0.805186   
        AutoFeat  Original  F1_score                   NaN                NaN   
                            AUC                        NaN                NaN   
                  Generated F1_score                   NaN                NaN   
                            AUC                        NaN                NaN   

                                       DecisionTreeClassifier  \
DataSet Framework Type      Evaluation                          
phoneme BigFeat   Original  F1_score                 0.866932   
                            AUC                      0.782632   
                  Generated F1_score                 0.871522   
                            AUC                      0.815942   
        AutoFeat  Original  F1_score                      NaN   
                            AUC                           NaN   
                  Generated F1_score                      NaN   
                            AUC                           NaN   

                                       ExtraTreesClassifier KNNClassifier  \
DataSet Framework Type      Evaluation                                      
phoneme BigFeat   Original  F1_score               0.936019      0.921094   
                            AUC                     0.89446      0.858644   
                  Generated F1_score               0.930966      0.915924   
                            AUC                    0.884627      0.842103   
        AutoFeat  Original  F1_score                    NaN           NaN   
                            AUC                         NaN           NaN   
                  Generated F1_score                    NaN           NaN   
                            AUC                         NaN           NaN   

                                             MLP RandomForestClassifier  \
DataSet Framework Type      Evaluation                                    
phoneme BigFeat   Original  F1_score    0.910813               0.857571   
                            AUC         0.849832               0.704181   
                  Generated F1_score    0.890332               0.855056   
                            AUC         0.783276               0.726091   
        AutoFeat  Original  F1_score         NaN                    NaN   
                            AUC              NaN                    NaN   
                  Generated F1_score         NaN                    NaN   
                            AUC              NaN                    NaN   

                                             SVM GradientBoostingClassifier  
DataSet Framework Type      Evaluation                                       
phoneme BigFeat   Original  F1_score    0.848415                   0.880342  
                            AUC         0.720213                   0.781564  
                  Generated F1_score    0.841986                   0.885335  
                            AUC         0.687313                   0.794159  
        AutoFeat  Original  F1_score         NaN                        NaN  
                            AUC              NaN                        NaN  
                  Generated F1_score         NaN                        NaN  
                            AUC              NaN                        NaN

In [ ]:
dataforSAFE(data_indices)

In [17]:
startPipeline(baseDir,datasets,ml_models,'AutoFeat')

C:\Users\kayak\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


62.415557861328125 seconds taken for the dataset phoneme.csv


Logistic_Regression AdaBoostClassifier  \
DataSet Framework Type      Evaluation                                          
phoneme BigFeat   Original  F1_score              0.837716               0.88   
                            AUC                    0.67018            0.78058   
                  Generated F1_score              0.835698           0.888627   
                            AUC                   0.659936           0.805186   
        AutoFeat  Original  F1_score              0.837716               0.88   
                            AUC                    0.67018            0.78058   
                  Generated F1_score              0.855198           0.884555   
                            AUC                    0.70893           0.792783   

                                       DecisionTreeClassifier  \
DataSet Framework Type      Evaluation                          
phoneme BigFeat   Original  F1_score                 0.866932   
                            AUC                      0.782632   
                  Generated F1_score                 0.871522   
                            AUC                      0.815942   
        AutoFeat  Original  F1_score                 0.866932   
                            AUC                      0.782632   
                  Generated F1_score                 0.861769   
                            AUC                      0.765106   

                                       ExtraTreesClassifier KNNClassifier  \
DataSet Framework Type      Evaluation                                      
phoneme BigFeat   Original  F1_score               0.936019      0.921094   
                            AUC                     0.89446      0.858644   
                  Generated F1_score               0.930966      0.915924   
                            AUC                    0.884627      0.842103   
        AutoFeat  Original  F1_score               0.936019      0.921094   
                            AUC                     0.89446      0.858644   
                  Generated F1_score               0.935319      0.900039   
                            AUC                    0.886768      0.817543   

                                             MLP RandomForestClassifier  \
DataSet Framework Type      Evaluation                                    
phoneme BigFeat   Original  F1_score    0.910813               0.857571   
                            AUC         0.849832               0.704181   
                  Generated F1_score    0.890332               0.855056   
                            AUC         0.783276               0.726091   
        AutoFeat  Original  F1_score    0.910813               0.857571   
                            AUC         0.849832               0.704181   
                  Generated F1_score    0.909019                0.86259   
                            AUC         0.848857               0.669631   

                                             SVM GradientBoostingClassifier  
DataSet Framework Type      Evaluation                                       
phoneme BigFeat   Original  F1_score    0.848415                   0.880342  
                            AUC         0.720213                   0.781564  
                  Generated F1_score    0.841986                   0.885335  
                            AUC         0.687313                   0.794159  
        AutoFeat  Original  F1_score    0.848415                   0.880342  
                            AUC         0.720213                   0.781564  
                  Generated F1_score    0.853556                   0.878447  
                            AUC         0.712293                   0.777827

In [20]:
dataforSAFE(data_indices)

In [25]:
for x,y in dataframe.iterrows():
    print(y)

Logistic_Regression           0.837716
AdaBoostClassifier                0.88
DecisionTreeClassifier        0.866932
ExtraTreesClassifier          0.936019
KNNClassifier                 0.921094
MLP                           0.910813
RandomForestClassifier        0.857571
SVM                           0.848415
GradientBoostingClassifier    0.880342
Name: (phoneme, BigFeat, Original, F1_score), dtype: object
Logistic_Regression            0.67018
AdaBoostClassifier             0.78058
DecisionTreeClassifier        0.782632
ExtraTreesClassifier           0.89446
KNNClassifier                 0.858644
MLP                           0.849832
RandomForestClassifier        0.704181
SVM                           0.720213
GradientBoostingClassifier    0.781564
Name: (phoneme, BigFeat, Original, AUC), dtype: object
Logistic_Regression           0.835698
AdaBoostClassifier            0.888627
DecisionTreeClassifier        0.871522
ExtraTreesClassifier          0.930966
KNNClassifier              